# Extrinsic Calibration

This notebook is an implementation, debugging and analysis of an Extrinsic Calibration Process using *CoppeliaSim* as a renderer. 

The goal is to implement and analyse, how all of the parameters and models used will affect the overall calibration and for how much. For this:
1. Make *CoppeliaSim's* clients that detects and sends blob coordinates detected in Vision Sensors' images;
2. Receive the messages asynchronously;
3. For each client, interpolate in time the detected blob positions;
4. Detect the calibration wand in the interpolated data;
5. Estimate the Fundamental and Essential Matrix, and it's scale;
6. Compute the relative pose between cameras;
7. Use this computation as initial guess to a bundle adjustment algorithm;

---

In [1]:
# Importing modules...
import numpy as np
import scipy as sp
import cv2
import socket

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.viewer3d import Viewer3D

from modules.vision.synchronizer import Synchronizer
from modules.vision.linear_projection import reprojection_error

from modules.integration.client import Client
from modules.integration.coppeliasim.server import CoppeliaSim_Server
from modules.integration.coppeliasim.camera import CoppeliaSim_Camera

# Instanciating `Server` and `Client` Structures

To wrap the information of all clients and mediate the communication between this notebook and *CoppeliaSim*, a `Server` object will be instanciated. 

The `Client` objects will be generated by their camera model and a synchronizer, representing their respective twin in the simulation.

For the `Server` instanciation, the following parameters must be given:
- Server address;
- *CoppeliaSim's* simulation Controller address;
- List containing all the clients present in the scene.

---

In [2]:
n_clients = 4 # Number of clients in the arena
clients = []  # Clients list

# Object matrix of Camera 0
base_matrix = np.array([[-7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 1.46327395e-13, -7.07106781e-01, -7.07106781e-01, 2.50000000e+00]])

# Create clients
for ID in range(n_clients):
    # Spread all cameras uniformely in a circle around the arena
    R = np.array(sp.spatial.transform.Rotation.from_euler('z', (360 / n_clients) * ID, degrees=True).as_matrix())
    pose = np.vstack((R @ base_matrix,
                      np.array([0, 0, 0, 1])))

    # Generate associated camera model
    camera = (CoppeliaSim_Camera(resolution=(1080, 1080), 
                                 fov_degrees=60.0,     
                                 pose=pose,
                                 distortion_model='fisheye',
                                 distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
                                 snr_dB=13
                                 ))
    
    clients.append(Client(camera=camera))

In [3]:
# Create server
server = CoppeliaSim_Server(clients=clients,
                            server_address=('127.0.0.1', 8888),
                            controller_address=('127.0.0.1', 7777))

# Requesting Scene

A **Scene Request** will send to *CoppeliaSim* the data necessary to instanciate the simulated clients' twins in the childscripts. The scene request can be used to reset the client's data from the server if called again.

---

In [4]:
# Request scene with the associated server clients
if not server.request_scene():
    sys.exit() # Scene request failed!

[SERVER] Wrapping up CoppeliaSim scene info
[SERVER] Scene info sent
[SERVER] Scene set!


# Requesting Capture

A **Capture Request** will trigger a simulation in *CoppeliaSim* sending the total simulation time of the requested capture. Once the simulation stops, another capture request can be called for another simulation

In the simulation start, the clients will be created and send their ID to the server for client registration.

---

In [5]:
# Capture specifications
blob_count = 3 # Number of expected markers
capture_time = 30.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(synchronizer):
    sys.exit() # Capture request failed!

# Wait for client identification
server.register_clients()

[SERVER] Capture info sent
[SERVER] Capture confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


# Running Simulation

The messages will be received here in the following loop until a server timeout is reached. To analyse the content of each message, toggle the `verbose` flag. 

The loop will wait for a message to be received by the socket. When a message comes, it will be recorded in their client's `message_log` and wait for the next message. 

Since the calibration prioritizes the amount of quality data and not real time triangulation, the in-loop actions will be post-processed to avoid message losses.

---

In [6]:
verbose = False

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


# Post-processing Data

For each client, the code will loop through it's message history and it will:
1. Decode message; 
2. Parse the message for it's contents;
3. Check if the message is valid;
    - A valid message is composed of a blob coodinate and it's area (per blob) and the PTS of the message.
4. Undistort blob data
5. Save data in the `Synchronizer` structure.

---

In [7]:
verbose = False

for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 
        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message

        # Extracting the message's PTS
        PTS = message[-1] # Last element of the message 

        # Valid message is [u, v, A] per blob and the PTS of the message
        if message.size !=  3 * blob_count + 1:

            if message.size == 1: # Only PTS
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        blob_data = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

        # Extracting centroids
        blob_centroids = blob_data[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(blob_data).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

# Matching Blobs to Markers

For the Fundamental Matrix estimation algorithm to work, points must be matched from an image to another. 

Since the Fundamental Matrix is still not known, it means the points cannot be ordered by epipolar lines. 

To work around that, the markers in the scene will be arranged in a collinear pattern, where the distance between adjacent markers in the line will be in an arbitrary non-unit ratio.

The reason for they to be collinear is the fact that in an undistorted image the distance ratio between the detected blobs will still be the same, but in pixels.

The ratio will make it possible to label each blob to each marker. The below algorithm will order the blobs according to the ratio in the following order:
1. The closest blob to the middle blob;
2. The middle blob;
3. The furthest blob from the middle blob;

---

In [8]:
def collinear_order(blobs, wand_ratio):
    # Distances between blobs
    distances = np.array([np.linalg.norm(blobs[0] - blobs[1]), 
                          np.linalg.norm(blobs[1] - blobs[2]), 
                          np.linalg.norm(blobs[2] - blobs[0])])
    
    # Shortest distance
    shortest = np.min(distances)

    # If x is invalid
    if shortest == 0.0 or shortest is np.nan:
        return None

    # Normalize distances
    distances /= shortest

    # Measured unique distance sums
    measured_unique_sums = np.array([distances[0] + distances[2],
                                     distances[0] + distances[1],
                                     distances[1] + distances[2]])
    
    # Exprected unique distance sums
    expected_unique_sums = np.array([wand_ratio[0] + wand_ratio[0] + wand_ratio[1],
                                     wand_ratio[0] + wand_ratio[1],
                                     wand_ratio[0] + wand_ratio[1] + wand_ratio[1]])
    
    # Error matrix
    difference_matrix = np.array([[np.abs(measured - expected)
                                   for measured in measured_unique_sums] 
                                   for expected in expected_unique_sums])

    # Using the hungarian (Munkres) assignment algorithm to find unique correspondences between blobs and epilines
    _, new_indices = sp.optimize.linear_sum_assignment(difference_matrix)

    return blobs[new_indices]

# Measured distances between markers
# Distances: [AB, BC, AC]
wand_distances = np.array([5e-2, 10e-2, 15e-2]) # In meters
wand_ratio = (1.0, wand_distances[1] / wand_distances[0])

# Playing Back a Camera's Feed

The following cell will replicate a real time camera feed of the simulation. Change the `ID` parameter to switch between camera views.

In this case, the blob labelling will be shown in the feed to check if the ordering algorithm is working as expected.

---

In [9]:
# Playback a camera image feed in fidelity time
playback = False

if playback:
    ID = 0 # Camera ID to be watched
        
    # Converting to np arrays
    sync_PTS = np.array(server.clients[ID].synchronizer.sync_PTS)
    blob_data = np.array(server.clients[ID].synchronizer.sync_blobs)
    
    # Generating frames
    images = []
    for PTS, blobs in zip(sync_PTS, blob_data):
        image = np.zeros(server.clients[ID].camera.image_shape)
        image = cv2.putText(image, str(PTS), 
                                    [40, 40], 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, 
                                    (255, 0, 0), 
                                    1, 
                                    cv2.LINE_AA)

        all_ordered_blobs_per_frame = collinear_order(blobs, (1, 2))

        if all_ordered_blobs_per_frame is not None:
            for tag, blob in zip(['A', 'B', 'C'], all_ordered_blobs_per_frame):

                image = cv2.putText(image, tag, 
                                    blob.astype(int), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    0.5, 
                                    (255, 255, 255), 
                                    1, 
                                    cv2.LINE_AA)
            
        images.append(image)

    # Getting delay between each frame
    delays = sync_PTS[1:] - sync_PTS[:-1]

    # Playing the animation
    for delay, image in zip(delays, images):
        cv2.imshow(f'Camera {ID} Feed', image)
        cv2.waitKey(int(1e3 * delay))

    # Closing all open windows 
    cv2.destroyAllWindows()

# Estimating Relative Poses

For each pair, the following process will be done to estimate their relative rotation and translation:
1. Order captured blobs in the proposed collinear order;
2. Estimate the Fundamental Matrix using OpenCV's function;
3. Calculate the Essential Matrix using using the estimated Fundamental Matrix;
4. Decompose the Essential Matrix into a Rotation Matrix and Unit Translation Vector;
5. Calculate the Projection Matrix of each matrix and triangulate the wand markers in each frame;
6. For each frame, relate the calculated unscaled marker distances to the measured marker distances and get the respective scale of the camera pair;
7. Build the estimated extrinsic matrix for each pair;

---

In [10]:
def decompose_essential_matrix(E, 
                               blobs_reference, 
                               blobs_auxiliary, 
                               intrinsic_matrix_reference, 
                               intrinsic_matrix_auxiliary):
    
    # Decompose the essential matrix
    R1, R2, t = cv2.decomposeEssentialMat(E)
    
    # Possible nase changes from the reference frame to
    R_t_options = [[R1,  t],
                   [R1, -t],
                   [R2,  t],
                   [R2, -t]]
    
    best_option = None

    # Projection matrix of the first camera
    P_reference = intrinsic_matrix_reference @ np.eye(4)[:3, :4]
    
    # Triangulate points and check their validity
    for option, R_t in enumerate(R_t_options):
        P_auxiliary = intrinsic_matrix_auxiliary @ np.hstack(R_t)

        triangulated_points_4D = cv2.triangulatePoints(P_reference.astype(np.float32), 
                                                       P_auxiliary.astype(np.float32), 
                                                       blobs_reference.T.astype(np.float32), 
                                                       blobs_auxiliary.T.astype(np.float32))
        
        # Normalize homogeneous coordinates and discard last row
        triangulated_points_3D = (triangulated_points_4D / triangulated_points_4D[-1])[:-1, :]

        # Count how many points have a positive z-coordinate 
        # This indicates if the point is in front of the camera or not
        frontal_points = np.count_nonzero(triangulated_points_3D[2] > 0)

        # No markers were shot behind the cameras
        if frontal_points == triangulated_points_3D.shape[1]:
            # Ambiguous decompostions found
            if best_option is not None:
                return None, None 
            
            best_option = option # Where all points are frontal points

    # No valid decomposition found
    if best_option is None:
        return None, None 
    
    # Valid decomposition
    return R_t_options[best_option]

In [11]:
# All possible camera pairs
client_ids = np.arange(n_clients)
pairs = [(i, j) for i in client_ids for j in client_ids[client_ids != i]] 

# Ordered blobs by collinearity
blob_data = [client.synchronizer.sync_blobs for client in server.clients]

all_ordered_blobs_per_frame = []
for same_frame_blobs in zip(*blob_data):
    ordered_blobs = [collinear_order(same_camera_blobs, wand_ratio) for same_camera_blobs in same_frame_blobs]

    # Only accept points where all points are visible in all views
    if any(O is None for O in ordered_blobs):
        continue

    all_ordered_blobs_per_frame.append(ordered_blobs)
all_ordered_blobs_per_frame = np.array(all_ordered_blobs_per_frame)

# Dicts for storing the final data
relative_poses = {}
extrinsic_matrices = {}
triangulated_markers = {}

for pair in pairs:
    # Getting data from pair
    reference, auxiliary = pair
    
    # Synchronized blobs for each camera in pair
    ordered_blobs_reference_per_frame = all_ordered_blobs_per_frame[:, reference, :, :]
    ordered_blobs_auxiliary_per_frame = all_ordered_blobs_per_frame[:, auxiliary, :, :]

    # Join all ordered blobs in a single matrix
    all_blobs_reference = np.vstack(ordered_blobs_reference_per_frame)
    all_blobs_auxiliary = np.vstack(ordered_blobs_auxiliary_per_frame)
    
    # Ground truth fundamental matrix
    F_ground_truth = server.multiple_view.fundamental_matrix[reference][auxiliary]

    F_estimated, mask = cv2.findFundamentalMat(points1=all_blobs_reference, 
                                               points2=all_blobs_auxiliary, 
                                               method=cv2.FM_8POINT)

    print(f'Pair {pair}:')
    print(f'> Matrix Distance of the Estimated F to the Ground Truth F = {np.abs(np.linalg.det(F_ground_truth - F_estimated)):.5e}')

    # Selecting inlier points    
    inlier_all_blobs_reference = all_blobs_reference[mask.ravel() == 1]
    inlier_all_blobs_auxiliary = all_blobs_auxiliary[mask.ravel() == 1]

    mask = np.array([np.prod(flags) for flags in mask.reshape(-1, 3)])

    inlier_blobs_reference_per_frame = ordered_blobs_reference_per_frame[mask == 1]
    inlier_blobs_auxiliary_per_frame = ordered_blobs_auxiliary_per_frame[mask == 1]

    # Calculating essential matrix
    E = server.clients[auxiliary].camera.intrinsic_matrix.T @ F_estimated @ server.clients[reference].camera.intrinsic_matrix

    # Decomposing essential matrix
    R, t = decompose_essential_matrix(E,
                                      inlier_all_blobs_reference,
                                      inlier_all_blobs_auxiliary,
                                      server.clients[reference].camera.intrinsic_matrix,
                                      server.clients[auxiliary].camera.intrinsic_matrix)

    # Check if decomposition worked
    if R is None and t is None:
        print('> Could not find reliable decomposition!')
        sys.exit() # Stop code execution

    # Calculating projection matrices
    # The reference camera will be the reference frame, thus the identity matrix
    P_reference = server.clients[reference].camera.intrinsic_matrix @ np.eye(4)[:3, :4]
    P_auxiliary = server.clients[auxiliary].camera.intrinsic_matrix @ np.hstack((R, t))
    
    all_triangulated_points = [] # List of triangulated points
    all_unscaled_distances = [] # List of all measured distance
    scales = [] # Scale factor for each triangulated frame

    for points_reference_per_frame, points_auxiliary_per_frame in zip(inlier_blobs_reference_per_frame, inlier_blobs_auxiliary_per_frame):
        triangulated_points_h = cv2.triangulatePoints(P_reference.astype(np.float32), 
                                                      P_auxiliary.astype(np.float32), 
                                                      points_reference_per_frame.T.astype(np.float32), 
                                                      points_auxiliary_per_frame.T.astype(np.float32))
        
        # Normalize homogeneous coordinates and discard last row
        triangulated_points = (triangulated_points_h / triangulated_points_h[-1])[:-1, :]

        # Unscaled distances
        unscaled_distances = np.array([np.linalg.norm(triangulated_points.T[0] - triangulated_points.T[1]),
                                       np.linalg.norm(triangulated_points.T[1] - triangulated_points.T[2]),
                                       np.linalg.norm(triangulated_points.T[0] - triangulated_points.T[2])])

        # Save values
        all_triangulated_points.append(triangulated_points)
        all_unscaled_distances.append(unscaled_distances)
        scales.append(np.sum(wand_distances) / np.sum(unscaled_distances))

    # Mean scale factor
    scale = np.mean(np.array(scales))
    print(f'> Mean K = {scale:.5f}')

    # Scaling data
    unscaled_distances_mean = np.mean(np.array(all_unscaled_distances), axis=0)
    scaled_distances_mean = unscaled_distances_mean * scale

    unscaled_distances_std = np.std(np.array(all_unscaled_distances), axis=0)
    scaled_distances_std = unscaled_distances_std * scale

    scaled_triangulated_points = np.hstack(np.array(all_triangulated_points)) * scale
    triangulated_markers[pair] = scaled_triangulated_points

    print(f'> Measured AB = {scaled_distances_mean[0]:.5f} m | Std deviation = {scaled_distances_std[0]:.5f} m')
    print(f'> Measured BC = {scaled_distances_mean[1]:.5f} m | Std deviation = {scaled_distances_std[1]:.5f} m')
    print(f'> Measured AC = {scaled_distances_mean[2]:.5f} m | Std deviation = {scaled_distances_std[2]:.5f} m')
    print()

    # Saving scaled matrices
    extrinsic_matrix_auxiliary = np.vstack((np.hstack((R, t * scale)),
                                            np.array([0, 0, 0, 1])))
    extrinsic_matrices[pair] = extrinsic_matrix_auxiliary
    relative_poses[pair] = np.linalg.inv(extrinsic_matrix_auxiliary)

# Update references
reference = 0 # The 0th camera will be the reference by default
server.multiple_view.camera_models[reference].update_extrinsic(np.eye(4))
for ID in client_ids[client_ids != reference]: 
   server.multiple_view.camera_models[ID].update_extrinsic(extrinsic_matrices[(reference, ID)])

Pair (0, 1):
> Matrix Distance of the Estimated F to the Ground Truth F = 8.79858e-17
> Mean K = 5.00714
> Measured AB = 0.05005 m | Std deviation = 0.00129 m
> Measured BC = 0.10004 m | Std deviation = 0.00204 m
> Measured AC = 0.15000 m | Std deviation = 0.00260 m

Pair (0, 2):
> Matrix Distance of the Estimated F to the Ground Truth F = 6.61074e-16
> Mean K = 7.07865
> Measured AB = 0.05022 m | Std deviation = 0.00172 m
> Measured BC = 0.10002 m | Std deviation = 0.00317 m
> Measured AC = 0.15011 m | Std deviation = 0.00485 m

Pair (0, 3):
> Matrix Distance of the Estimated F to the Ground Truth F = 1.61356e-15
> Mean K = 5.05416
> Measured AB = 0.04994 m | Std deviation = 0.00159 m
> Measured BC = 0.10012 m | Std deviation = 0.00205 m
> Measured AC = 0.15003 m | Std deviation = 0.00270 m

Pair (1, 0):
> Matrix Distance of the Estimated F to the Ground Truth F = 8.79858e-17
> Mean K = 5.00715
> Measured AB = 0.05005 m | Std deviation = 0.00129 m
> Measured BC = 0.10004 m | Std devia

# Displaying Calibrated Camera Poses

To visually inspect if the results are coherent to the scene, the calibrated camera poses will be displayed in a 3D viewer. The 0th camera will be shown as the reference frame.

Choose the Pair ID to show the respective triangulated points of each pair.

---

In [12]:
# Create the Scene Viewer
scene = Viewer3D(title='Calibrated Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

for pair in pairs:
    # Add triangulated markers to the scene
    scene.add_points(triangulated_markers[pair], f'Pair {pair}')

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Performing Bundle Adjustment

The above calibration procedure is not the best converging one, since each pair is calibrated separately. This means that the triangulation of a 3D point in the scene is most likely to not be coherent between camera pairs, which severely impacts not only on the accuracy and precision of the triangulated marker positions.

When dealing with control systems, this can lead to harmful instabilities on the system.

To deal with that, a non-linear optimization process called **Bundle Adjustment** is done to get a statistically better result on the calibration, optimizing the points position and camera parameters globally to converge to a better result.

The key idea is that the triangulated point projected point in the image plane should be as close as possible to the observed point on the image plane. To ensure this, a cost function regarding the total reprojection error is made and minimized through the Levenberg–Marquardt algorithm.

The cost function follows:

$$R_e(C, M_{3\times 1}, b_{2\times 1}) := |p(C, M_{3\times 1}) - b_{2\times 1}|$$

Where:
- $R_e$ characterizes the reprojection error function;
- The function $p$ represents a projection transformation;
- $C$ is a vector that characterizes the camera intrinsic and extrinsic parameters;
- $M$ is the spatial position of a marker in the scene;
- $b$ is a detected blob position.

And the bundle adjustment process optimizes $M_i$ and $C_j$ as:

$$\argmin_{M_i, C_j}\sum_{i=1}^{n}\sum_{j=1}^{v} R_e(C_j, M_i, b_{ij})^2$$

Where:
- There are $n$ 3D Markers;
- There are $v$ views;
- The $j$ th camera is parameterized by a vector $C_j$;
- Each $i$ th 3D point estimate is $M_i$;
- $b_{ij}$ is the 2D view of the $i$ th point on the $j$ th camera.

---

In [13]:
# All pairs with no repetition
client_ids = np.arange(n_clients)
unique_pairs = [(i, j) for i in client_ids for j in client_ids[i+1:]]

# Triangulate points for each pair
all_triangulated_points = []
for pair in unique_pairs:
    reference, auxiliary = pair

    triangulated_points = []
    for blobs_in_frame in all_ordered_blobs_per_frame:
        triangulated_points.append(server.multiple_view.triangulate_by_pair(pair, [blobs_in_frame[reference],
                                                                                   blobs_in_frame[auxiliary]]))

    triangulated_points = np.hstack(np.array(triangulated_points))
    all_triangulated_points.append(triangulated_points)
    
all_triangulated_points = np.array(all_triangulated_points)
all_ordered_blobs = np.array([np.vstack(np.array(all_ordered_blobs_per_frame)[:, ID]) for ID in client_ids])

# Relation between all observations in the capture and the chosen ones
total_observations = all_ordered_blobs.shape[1]
n_observations = 72

# Get roughly equally spaced observations in time (and hopefully in space)
indexes = np.arange(0, total_observations, total_observations//n_observations + 1)
pairs_sequence = np.arange(n_observations) % len(unique_pairs)
all_ordered_blobs = all_ordered_blobs[:, indexes]
all_triangulated_points = np.hstack([all_triangulated_points[p, :, i].reshape(3, -1) for p, i in zip(pairs_sequence, indexes)])

In [14]:
def total_reprojection_error(optimize, intrinsic_matrices, all_ordered_blobs):
    # Retrieve parameters
    n_cameras = len(intrinsic_matrices)
    rvecs_tvecs = optimize[:n_cameras * 6].reshape(n_cameras, 2, 3) # 6 DoF per Camera
    all_triangulated_points = optimize[n_cameras * 6:].reshape(3, -1)

    # Calculate new projection matrices
    extrinsic_matrices = [np.hstack((cv2.Rodrigues(rvec)[0], tvec.reshape(3, -1))) for rvec, tvec in rvecs_tvecs]
    projection_matrices = [I @ E for I, E in zip(intrinsic_matrices, extrinsic_matrices)]

    # Compute residuals
    residuals = []
    for projection_matrix, all_detected_blobs in zip(projection_matrices, all_ordered_blobs):
        camera_residuals = reprojection_error(points_to_project=all_triangulated_points, 
                                              points_in_image=all_detected_blobs,
                                              projection_matrix=projection_matrix)
            
        residuals.append(camera_residuals)

    residuals = np.concatenate(residuals)
    
    return residuals

In [15]:
# Condensing initial guess into initial guess parameter vector
rvecs_tvecs = np.array([[cv2.Rodrigues(camera.extrinsic_matrix[:3, :3])[0].flatten(), 
                         camera.extrinsic_matrix[:3, -1]] for camera in server.multiple_view.camera_models])

initial_guess = np.hstack((rvecs_tvecs.flatten(), all_triangulated_points.flatten()))

# Optimizing parameter vector by minimizing cost function using Levenberg–Marquardt algorithm
result = sp.optimize.least_squares(fun=total_reprojection_error, 
                                   x0=initial_guess,
                                   method='lm',
                                   args=([camera.intrinsic_matrix for camera in server.multiple_view.camera_models],
                                         all_ordered_blobs))

optimized_parameters = np.array(result.x)

# Retrieving data from optimized parameter vector
rvecs_tvecs_adjusted = optimized_parameters[:server.n_clients * 6].reshape(server.n_clients, 2, 3)
adjusted_extrinsics = [np.vstack((np.hstack((cv2.Rodrigues(rvec)[0], tvec.reshape(3, -1))), [0, 0, 0, 1])) for rvec, tvec in rvecs_tvecs_adjusted]
adjusted_poses = [np.linalg.inv(E) for E in adjusted_extrinsics]

all_triangulated_points_adjusted = optimized_parameters[server.n_clients * 6:].reshape(3, -1)

# Optimal cost ~ Number of observations * Variance
optimal_cost = n_observations * n_clients # For a std deviation of 1 px

# Show data
print(f'Optimal cost at the solution: {optimal_cost:.2f}')
print(f'Actual cost at the solution: {result.cost:.2f}')
print(f'Standard Deviation: {np.sqrt(result.cost / optimal_cost):.2f} px')

Optimal cost at the solution: 288.00
Actual cost at the solution: 273.18
Standard Deviation: 0.97 px


In [16]:
# Create the Scene Viewer
scene = Viewer3D(title='Bundle Adjusted Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, adjusted_pose in enumerate(adjusted_poses): 
    scene.add_frame(adjusted_pose, f'Camera {ID}', axis_size=0.4)

# Add triangulated markers to the scene
scene.add_points(all_triangulated_points_adjusted, f'Optimized Points')

# Plot scene
scene.figure.show(renderer='notebook_connected')